### Face Detection and Tracking

In [1]:
# Importing needed packages
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import time

In [2]:
# Getting name of haar cascade files
files = os.listdir(cv2.data.haarcascades)

for file in files:
    print(file)

haarcascade_upperbody.xml
haarcascade_frontalcatface_extended.xml
haarcascade_profileface.xml
haarcascade_frontalcatface.xml
haarcascade_frontalface_alt2.xml
haarcascade_eye.xml
__init__.py
haarcascade_lefteye_2splits.xml
__pycache__
haarcascade_frontalface_alt_tree.xml
haarcascade_righteye_2splits.xml
haarcascade_frontalface_alt.xml
haarcascade_lowerbody.xml
haarcascade_russian_plate_number.xml
haarcascade_frontalface_default.xml
haarcascade_smile.xml
haarcascade_license_plate_rus_16stages.xml
haarcascade_fullbody.xml
haarcascade_eye_tree_eyeglasses.xml


In [3]:
# Creating our face detector
faceDetector = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

In [4]:
# Function to detect faces
def detectFace(img, blur=False, draw=True):
    detections = img.copy()
    faces = faceDetector.detectMultiScale(detections, scaleFactor = 1.1, minNeighbors = 7, minSize=(240,240), maxSize=(700,700)) 

    # Drawing rectangle
    if draw:
        for (x,y,w,h) in faces:
            cv2.rectangle(detections, (x,y), (x+w, y+h), (255,0,255), 6) # purple rectangle for haar face detection mode
            cv2.putText(detections, "Facial Detection Using HAAR" + (" (Blurred)" if blur else ""), (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,255), 3)
            if blur:
                detections[y:y+h, x:x+w] = cv2.GaussianBlur(detections[y:y+h, x:x+w], (41, 41), 0)
    return detections, faces 


In [5]:
# Function to track faces
def trackFace(img, tracker, blur=False):
    ok, bbox = tracker.update(img)
    if ok:
        (x,y,w,h) = [int(v) for v in bbox]
        cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0), 6) # green rectangle for csrt mode
        cv2.putText(img, "Facial Tracking Using CSRT" + (" (Blurred)" if blur else ""), (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 3)
        if blur:
            img[y:y+h, x:x+w] = cv2.GaussianBlur(img[y:y+h, x:x+w], (41, 41), 0)
    return img, ok

### Connecting to the webcam

In [6]:
capture = cv2.VideoCapture(0) # 0 is the code for hardware camera

# flag for grayscale
gs_mode = False
blur_mode = False
edge_mode = False
csrt_mode = False

# tracker for csrt mode
tracker = None

# Endless loop (use break to escape)
while True:
    ret, frame = capture.read()

    time.sleep(1/60) # 60 fps (adjust as needed)

    # Store which key was pressed (using several if statements with waitkey caused issues)
    key = cv2.waitKey(1) & 0xFF

    # Toggle grayscale mode
    if key == ord('g'):
        gs_mode = not gs_mode # simple trick to toggle booleans
    
    # Toggle edge mode
    if key == ord('e'):
        edge_mode = not edge_mode # simple trick to toggle booleans
    
    # Toggle blur mode
    if key == ord('b'):
        blur_mode = not blur_mode # simple trick to toggle booleans

    # Toggle csrt mode
    if key == ord('c'):
        csrt_mode = not csrt_mode
        # enable/init tracker
        if csrt_mode:
            _, faces = detectFace(frame, blur=blur_mode, draw=False)
            (x,y,w,h) = faces[0]
            tracker = cv2.TrackerCSRT_create()
            tracker.init(frame, (x,y,w,h))

    # Using canny for edge mode
    if edge_mode:
        frame = cv2.Canny(frame, 30, 150)
        cv2.putText(frame, "Edge Detection Using Canny", (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,255), 3)
            
    if csrt_mode and tracker is not None:
        frame, tracking_ok = trackFace(frame, tracker, blur=blur_mode)
        if not tracking_ok:
            csrt_mode = False # turn if it fails just in case
            tracker = None # disable tracker as well

    # update frame value for grayscale mode
    if gs_mode:
        
        # convert frame to grayscale
        frame, _ = detectFace(frame, blur=blur_mode, draw=(csrt_mode == False))
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        
        # Using cv2.cvtColor() is generally more efficient
        # frame = frame[:,:,1]    
    elif not csrt_mode:
        frame, _ = detectFace(frame, blur=blur_mode, draw=True) # dont need seperate if for blur mode
        

    cv2.imshow('Face Detection', frame)

    if key == ord('q'):
        break

capture.release()
cv2.destroyAllWindows() # sometimes has issues on mac, let me know if causes issue on windows
cv2.destroyWindow('Face Detection')

2024-10-31 01:14:39.267 Python[43638:4528148] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
2024-10-31 01:14:40.986 Python[43638:4528148] +[IMKClient subclass]: chose IMKClient_Legacy
2024-10-31 01:14:40.986 Python[43638:4528148] +[IMKInputSession subclass]: chose IMKInputSession_Legacy
